# 类代码编写细节

In [1]:
class Shareddata:
    data = 30

In [2]:
x = Shareddata()
y = Shareddata()

In [3]:
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

(30, 4419250064, 30, 4419252304, 30, 4336330872)

In [4]:
x.data = 80
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

(80, 4419250064, 30, 4419252304, 30, 4336330872)

对实例的属性进⾏赋值运算会在该实例内创建或修改名称，⽽不是在共享的类中

In [5]:
Shareddata.data = 99

In [6]:
x.data, id(x), y.data, id(y), Shareddata.data, id(Shareddata.data)

(80, 4419250064, 99, 4419252304, 99, 4336333080)

In [7]:
class SharedList:
    data = [1, 2, 3]

In [8]:
x = SharedList()
y = SharedList()
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

([1, 2, 3], 4419274064, [1, 2, 3], 4419250064, [1, 2, 3], 4419298688)

In [9]:
x.data = [4, 5, 6]
y.data = [1, 2, 3]  # 赋值相同的值也会改变引用
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

([4, 5, 6], 4419274064, [1, 2, 3], 4419250064, [1, 2, 3], 4419298688)

In [10]:
SharedList.data = ['abc']

In [11]:
x.data, id(x), y.data, id(y), SharedList.data, id(SharedList.data)

([4, 5, 6], 4419274064, [1, 2, 3], 4419250064, ['abc'], 4419254080)

# 修改实例属性

In [12]:
class Mixname:
    data = 'spam'

    def __init__(self, name):
        self.data = name

    def display(self):
        print(self.data, Mixname.data)

In [13]:
x = Mixname('eggs')
y = Mixname('apple')
x.display(), y.display()

eggs spam
apple spam


(None, None)

In [14]:
Mixname.display(x)

eggs spam


In [15]:
Mixname.display()

TypeError: Mixname.display() missing 1 required positional argument: 'self'

# 继承

In [ ]:
class Super:
    def method(self):
        print('in Super.method')

In [ ]:
class Sub(Super):
    def method(self):
        print('in Sub.method')
        Super.method(self)
        print('ending Super.method')

In [ ]:
x = Super()
x.method()

In [ ]:
x = Sub()
x.method()

In [ ]:
class Super:

    def method(self):
        print('in Super.method')

    def delegate(self):
        self.action()


class Inheritor(Super):
    # Default behavior # Expected to be defined # Inherit method verbatim
    pass


class Replacer(Super):

    def method(self):
        # Replace method completely
        print('in Replacer.method')


class Extender(Super):

    def method(self):
        # Extend method behavior
        print('starting Extender.method')
        Super.method(self)
        print('ending Extender.method')


class Provider(Super):

    def action(self):
        # Fill in a required method
        print('in Provider.action')


In [ ]:
for klass in (Inheritor, Replacer, Extender):
    print('\n' + klass.__name__ + '...')
    klass().method()
    print('\nProvider...')
    x = Provider()
    x.delegate()
    print('-' * 50)

In [ ]:
class Super:

    def method(self):
        print('in Super.method')

    def delegate(self):
        self.action()

    def action(self):
        raise NotImplementedError('action must be defined')


In [ ]:
x = Super()
x.delegate()

In [ ]:
y = Provider()
y.delegate()

In [ ]:
class Sub(Super):
    def action(self):
        print('in Sub.action')

In [ ]:
z = Sub()
z.delegate()

# 抽象类

In [ ]:
from abc import ABCMeta, abstractmethod


class Super(metaclass=ABCMeta):

    def delegate(self):
        self.action()

    @abstractmethod
    def action(self):
        pass

In [ ]:
x = Super()

In [ ]:
class Sub(Super):
    pass

In [ ]:
y = Sub()

In [ ]:
class Sub(Super):
    def action(self):
        print('in Sub.action')


In [ ]:
z = Sub()
z.delegate()

## 命名空间

In [16]:
X = 11


def f():
    print(X)


def g():
    # Global (module) name/attribute (X, or manynames.X)
    # Access global X (11)
    X = 22
    # Local (function) variable (X, hides module X)
    print(X)


class C:
    X = 33

    def m(self):
        # Class attribute (C.X)
        X = 44
        self.X = 55

# Local variable in method (X) # Instance attribute (instance.X)

In [17]:
print(X)
f()
g()
print(X)
print('-' * 50)
obj = C()
print(obj.X)
obj.m()
print(obj.X)
print(C.X)

11
11
22
11
--------------------------------------------------
33
55
33


In [18]:
X = 1


def nester():
    print(X)

    class C:
        print(X)

        def method1(self):
            print(X)

        def method2(self):
            X = 3
            print(X)

    # Global: 1 # Global: 1 # Global: 1 # Hides global # Local: 3

    I = C()  # 1
    I.method1()  # 1
    I.method2()  # 3


print(X)  # 1
print('-' * 40)
nester()  # 1 1 1 3

1
----------------------------------------
1
1
1
3


In [19]:
X = 1


def nester():
    X = 2
    print(X)

    class C:
        X = 3
        print(X)

        def method1(self):
            print(X)  # 2 not 3
            print(self.X)  # 3

        def method2(self):
            X = 4
            print(X)  # 4
            self.X = 5
            print(self.X)  # 5

    I = C()  # 3
    I.method1()  # 2 3
    I.method2()  # 4 5


print(X)  # 1
print('-' * 40)
nester()  # 3 2 3 4 5

1
----------------------------------------
2
3
2
3
4
5


In [20]:
X = 1

def nester():
    X = 2  # Hides global 

    print(X)  # Local: 2

    class C:
        X = 3  # Class local hides nester's: C.X or I.X (not scoped) 
        print(X)  # Local: 3

        def method1(self):
            print(X)  # In enclosing def (not 3 in class!): 2
            print(self.X)  # Inherited class local: 3

        def method2(self):
            X = 4  # Hides enclosing (nester, not class)
            print(X)  # Local: 4
            self.X = 5  # Hides class
            print(self.X)  # Located in instance: 5

    I = C()
    I.method1()
    I.method2()


print(X)
print('-' * 40)
nester()

1
----------------------------------------
2
3
2
3
4
5


In [21]:
class Sup:
    def hello(self):
        self.data1 = 'spam'
        
class Sub(Sup):
    def hola(self):
        self.data2 = 'eggs'

In [22]:
x = Sub()

In [23]:
x.__dict__

{}

In [24]:
x.__class__

__main__.Sub

In [25]:
Sub.__base__

__main__.Sup

In [26]:
Sup.__base__

object

In [27]:
y = Sub()
y.hello()
y.__dict__

{'data1': 'spam'}

In [28]:
y.hola()
y.__dict__

{'data1': 'spam', 'data2': 'eggs'}

In [29]:
list(Sub.__dict__)

['__module__', 'hola', '__doc__']

In [30]:
list(Sup.__dict__)

['__module__', 'hello', '__dict__', '__weakref__', '__doc__']

In [31]:
y.data1, y.__dict__['data1']

('spam', 'spam')

In [32]:
y.data3 = 'toast'
y.__dict__

{'data1': 'spam', 'data2': 'eggs', 'data3': 'toast'}

In [33]:
y.__dict__['hello']

KeyError: 'hello'

In [34]:
Sup.__base__

object

In [35]:
Sup.__base__.__base__